In [198]:
import pandas as pd
import matplotlib as plt
import datetime
import numpy as np
import os

pd.options.display.max_rows = 200
filename = 'KÄYTTÖTILI FI52 1011 3500 7285 21 - 2023-10-27 14.09.21.csv'
sep = ";"

In [199]:
def get_raw_data(seperator,filename):
    raw_data = pd.read_csv(filename,sep=seperator)
    raw_renamed = raw_data.rename(columns={"Kirjauspäivä": "date", "Määrä": "amount","Otsikko":"subject"})
    raw_renamed['amount'] = [np.float64(n.replace(",",".")) for n in raw_renamed['amount']]
    raw_renamed.loc[raw_renamed["date"] == "Varaus","date"] = datetime.datetime.now().date().strftime("%Y/%m/%d")
    raw_renamed.date = pd.to_datetime(raw_renamed.date)
    date_fixed = raw_renamed[["date","amount","subject"]].set_index("date").copy()
    date_fixed = date_fixed.loc[date_fixed["amount"] < 0,:]
    date_fixed['amount'] = date_fixed['amount'].abs()
    return date_fixed

In [200]:
data = get_raw_data(sep,filename)

In [201]:
def filter_values(df):
    df["category"] = None
    df.loc[(df["amount"] == 8.80 )| (df["subject"].str.contains("VAPE")),"category"]= "vape liquid"
    df.loc[df["subject"].str.contains("VR"),"category"] = "train"
    df.loc[df["subject"].str.contains("HSL|Nysse|TAMPEREEN KAUPUNK", regex=True),"category"] = "public transport" 
    df.loc[(df["subject"].str.contains("KAMMONEN"))|(df["amount"] == 631.0 ),"category"] = "rent"
    df.loc[(df["subject"].str.
      contains("Netflix.com|STEEL LS OY|a.NordVPNnl|STEELFEST|Varusteleka|AUSSIE BAR|TAMPEREEN MUSII|AWS EMEA|Spotify")),"category"] = "non-essential"
    df = df.loc[~(df["subject"].str.contains("SUNDAY"))]
    df.loc[df["category"].isnull(),"category"] = "living expense"
    return df

In [202]:
filtered = filter_values(data)

In [205]:
filtered.loc[(filtered["category"] == "living expense")]

,amount,subject,category
date,,,
2023-10-27,0.95,Prisma Kerava,living expense
2023-10-27,6.95,Marmaris Market,living expense
2023-10-27,6.60,24 Pesula Kerava,living expense
2023-10-27,2.78,K-Market Ahjontori,living expense
2023-10-27,10.34,Prisma Kerava,living expense
2023-10-27,3.00,MCDONALD'S 68,living expense
2023-10-27,3.74,Lidl TRE-Hervanta,living expense
2023-10-26,3.90,LIDL TAMPERE HERVANTA MN2,living expense
2023-10-26,2.78,K-market Amuri,living expense


In [195]:
filtered.groupby("category").aggregate({"amount":"sum"})

,amount
category,
living expense,539.69
non-essential,489.17
public transport,210.10
rent,871.00
train,108.70
vape liquid,76.39


In [197]:
filtered.groupby(filtered.index.isocalendar().week).aggregate({"amount":"sum"})

,amount
week,
40,1102.69
41,366.34
42,345.69
43,480.33
